In [2]:
import numpy as np
import pandas as pd

import cv2
import mediapipe as mp

import utlis



In [3]:
# get image names (e.g: image00002)
img_names = utlis.get_img_names('./AFLW2000')


In [4]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=1)

with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2) as face_mesh:
    
    # will contain info for all images
    data = []
    
    for img_name in img_names:
        img_path = f'./AFLW2000/{img_name}.jpg'
        mat_path = f'./AFLW2000/{img_name}.mat'
        
        # read image
        img = cv2.imread(img_path)

        # convert image to RGB (for mediapipe usage)
        img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # get face landmarks for each detected face
        results = face_mesh.process(img_RGB)
        
        # will contain image name, 468 landmarks and (roll, pitch, yaw) for each image
        row = {}
        
        # check if only one face is detected
        if results.multi_face_landmarks and len(results.multi_face_landmarks)==1:
            row['img_name'] = img_name
            
            # loop through detected faces
            for face_landmarks in results.multi_face_landmarks:

                # loop over 468 landmarks
                for idx, lm in enumerate(face_landmarks.landmark):
                    row[f'x{idx+1}'] = lm.x
                    row[f'y{idx+1}'] = lm.y
                
                # get yaw, pitch, roll from .mat files
                yaw, pitch, roll = utlis.get_ypr_from_mat(mat_path)
                
                row['yaw'] = yaw
                row['pitch'] = pitch
                row['roll'] = roll
                
                data.append(row)
                
df = pd.DataFrame(data)




In [5]:
df

,img_name,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,y465,x466,y466,x467,y467,x468,y468,yaw,pitch,roll
0,image00002,0.485651,0.686891,0.489194,0.639722,0.489803,0.647257,0.479730,0.562529,0.490558,...,0.492813,0.541429,0.500371,0.665669,0.471336,0.676091,0.464005,0.018227,-0.399231,0.085676
1,image00004,0.441502,0.642091,0.417273,0.592251,0.448839,0.610546,0.435994,0.545069,0.415757,...,0.511724,0.500658,0.514913,0.584451,0.500244,0.594060,0.495387,1.189533,0.470065,0.300959
2,image00008,0.503922,0.694309,0.469521,0.653913,0.489504,0.658751,0.444433,0.590285,0.461756,...,0.510606,0.490586,0.518652,0.595615,0.455611,0.602670,0.444623,0.299208,-0.175379,-0.373374
3,image00013,0.510238,0.669038,0.492729,0.604797,0.499756,0.624468,0.467368,0.542362,0.488200,...,0.488652,0.505667,0.492681,0.602484,0.452699,0.611310,0.448576,0.011965,-0.026812,-0.220662
4,image00014,0.492562,0.659880,0.484423,0.589555,0.490822,0.613645,0.471540,0.528240,0.483233,...,0.491427,0.522413,0.492968,0.626868,0.476350,0.637145,0.471961,0.110732,0.057119,-0.043283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,image04345,0.515443,0.683951,0.532837,0.625785,0.518815,0.639885,0.511650,0.560010,0.535343,...,0.503230,0.540073,0.508401,0.629060,0.479723,0.635765,0.474628,-0.283822,-0.306358,0.038554
1831,image04348,0.543907,0.689845,0.565286,0.635285,0.549288,0.646834,0.545584,0.562769,0.568493,...,0.500255,0.574974,0.506461,0.659800,0.479817,0.667754,0.470887,-0.429723,-0.367547,0.122791
1832,image04356,0.586373,0.680103,0.604407,0.627700,0.593793,0.645848,0.602947,0.568153,0.609593,...,0.539478,0.648903,0.541426,0.734017,0.547668,0.743284,0.540139,0.567114,-0.156035,-0.108536
1833,image04358,0.491617,0.692910,0.500626,0.637631,0.497282,0.649833,0.490064,0.559349,0.503418,...,0.496594,0.550279,0.502602,0.673963,0.481876,0.684592,0.474287,-0.070430,-0.197102,0.105118


In [7]:
# Create a directory 'Data' if it doesnot exists 
out_dir = './Data'

# Save data to csv
df.to_csv(f'{out_dir}/Data.csv',  index=False)


## Normalize Data

In [8]:
landmarks = df.iloc[:, 1:-3]

nose_x = landmarks['x2']
nose_y = landmarks['y2']
chin_x = landmarks['x429']
chin_y = landmarks['y429']

# subtract nose point from all points and divide by the distance between nose and the chin
lands_normalized = utlis.normalize_df(landmarks, nose_x, nose_y, chin_x, chin_y)

df_normalized = df.copy()
df_normalized.iloc[:, 1:-3] =  lands_normalized

In [10]:
df_normalized

,img_name,x1,y1,x2,y2,x3,y3,x4,y4,x5,...,y465,x466,y466,x467,y467,x468,y468,yaw,pitch,roll
0,image00002,-0.021071,0.280501,0.0,0.0,0.003618,0.044809,-0.056280,-0.459042,0.008113,...,-0.873625,0.310625,-0.828678,1.049446,-1.001342,1.111425,-1.044938,0.018227,-0.399231,0.085676
1,image00004,0.152873,0.314467,0.0,0.0,0.199169,0.115431,0.118121,-0.297697,-0.009566,...,-0.508087,0.526120,-0.487966,1.054817,-0.580519,1.115447,-0.611167,1.189533,0.470065,0.300959
2,image00008,0.205183,0.240938,0.0,0.0,0.119186,0.028852,-0.149639,-0.379511,-0.046317,...,-0.854755,0.125639,-0.806765,0.752087,-1.182776,0.794164,-1.248309,0.299208,-0.175379,-0.373374
3,image00013,0.090869,0.333388,0.0,0.0,0.036468,0.102085,-0.131615,-0.324014,-0.023502,...,-0.602754,0.067143,-0.581845,0.569590,-0.789336,0.615394,-0.810732,0.011965,-0.026812,-0.220662
4,image00014,0.040906,0.353475,0.0,0.0,0.032162,0.121086,-0.064759,-0.308188,-0.005986,...,-0.493225,0.190950,-0.485478,0.715971,-0.569004,0.767628,-0.591064,0.110732,0.057119,-0.043283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,image04345,-0.119273,0.398858,0.0,0.0,-0.096150,0.096688,-0.145282,-0.451030,0.017186,...,-0.840384,0.049623,-0.804922,0.659827,-1.001575,0.705805,-1.036511,-0.283822,-0.306358,0.038554
1831,image04348,-0.135242,0.345140,0.0,0.0,-0.101198,0.073061,-0.124628,-0.458729,0.020289,...,-0.854186,0.061285,-0.814926,0.597888,-0.983471,0.648205,-1.039959,-0.429723,-0.367547,0.122791
1832,image04356,-0.105000,0.305106,0.0,0.0,-0.061799,0.105663,-0.008501,-0.346707,0.030192,...,-0.513668,0.259070,-0.502324,0.754643,-0.465979,0.808595,-0.509819,0.567114,-0.156035,-0.108536
1833,image04358,-0.055918,0.343104,0.0,0.0,-0.020752,0.075734,-0.065553,-0.485869,0.017330,...,-0.875375,0.308179,-0.838083,1.075851,-0.966720,1.141824,-1.013826,-0.070430,-0.197102,0.105118


In [11]:
# Create a directory 'Data' if it doesnot exists 
out_dir = './Data'

# Save data to csv
df_normalized.to_csv(f'{out_dir}/Data_normalized.csv',  index=False)
